In [1]:
import sys, os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.signal
import importlib
import h5py
sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [2]:
outfile='/Users/ajc/Core/PostProcessing/2020/barrow_watershed/projections_02/Nov10/'

In [3]:
#path_cades = "/Users/ajc/FUSE/simulations/tests/ats-regression-tests/08_energy/"

path ='/Users/ajc/FUSE/simulations/tests/ats-intermediate-ngee-up/single_column/'
var_set = ['surface_flux-day.dat']

In [4]:
S1 = [os.path.join(path, 'test0') ] 
Simulations = [S1]
#print (Simulations[0],)


In [5]:
mol_density = 55000.
to_mm = 1000.
to_day_mm = 86400 * 1000
to_day_year_mm = 86400 * 365 * 1000

In [6]:
def read_file(file):
    d = np.loadtxt(file,usecols=1,skiprows=6)
    t1 = np.loadtxt(file,usecols=0,skiprows=6)
    t = np.array([x/86400./365. for x in t1])
    #print (len(d),len(t))
    l = np.minimum(len(d),len(t))
    return d[:l],t[:l]

def read_set(set,var):
    dat = []
    tim = []
    for s in set:
        file = os.path.join(path_cades,s, var)
        #print (file)
        d = np.loadtxt(file,usecols=1,skiprows=6)
        t1 = np.loadtxt(file,usecols=0,skiprows=6)
        t = np.array([x/86400./365. for x in t1])
        
        L = len(d)%365
        if L >0:
            d = d[:-L]
            t = t[:len(d)]
            
        dat.append(d)
        tim.append(t)
    dat = np.concatenate(dat)
    tim = np.concatenate(tim)
    print ('Lengths: ',len(dat)%365, len(tim)%365,L)
    
    return dat, tim

In [7]:
#forcing_data = h5py.File(os.path.join(path,'data/spinup-10yr.h5'),'r')
forcing_data = h5py.File("/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2015-FM_2006.h5",'r')

In [8]:
d1 = forcing_data['precipitation rain [m s^-1]']
d2 = forcing_data['precipitation snow [m SWE s^-1]']
rain_mm = np.reshape(d1,(-1,365)).mean(axis=1)*86400*365*1000
snow_mm = np.reshape(d2,(-1,365)).mean(axis=1)*86400*365*1000
precip_annual = snow_mm + rain_mm
r1 = np.array(d1)*86400*1000
r2 = np.array(d2)*86400*1000
precip_daily = r1 + r2

In [9]:
#key_var = ['surface-evaporative_flux','surface-depth','surface_star-carbon_decomposition_rate']
key_var = ['surface_star-evaporative_flux','surface-depth','surface_star-carbon_decomposition_rate']

In [10]:

def getSurfaceData(i,var,hdf_file='ats_vis_surface_data.h5',average=True):
    Data = []
    print ('getSurfaceData: ',var,average)
    for s in Simulations[i]:
        infile = os.path.join(s,hdf_file)
        print (infile)
        dx=h5py.File(infile,'r')
        
        d = np.array(readdata.GetSurfVarFromVis_All(infile,var)[var])
        if (average):
            d = d.mean(axis=1)
        else:
            d = d.sum(axis=1)
        L = len(d)%365
        if L >0:
            d = d[:-L]
            
        Data.append(d)
        
    d = dict()
    d[var] = np.concatenate(Data)
    
    return d

In [11]:
colors = ['r','k','g','m','c','y']
marks = ['.','*', '+', 'o']
marks = ['-','-','-']
Labels = ['basecase','dynamic_sg']

In [12]:
Slabels = ['S1', 'S6']
surface_area = 1#2.625

In [13]:
def Evaporation():
    #%matplotlib qt
    #fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    #fig.subplots_adjust(hspace =.02, wspace=1.02)

    #plt.tight_layout()
    #axs = axs.ravel()
    Colors = ['k','b','grey','g','r','m']
    time = 2006 + np.linspace(0,94,94,dtype='int')
    d3_sf = []
    sf_change = 0
    for i,sim in enumerate(Simulations):
        var = key_var[0]
        d1 = getSurfaceData(i, var,hdf_file='ats_vis_surface_star_data.h5')[var]
        d2 = [x*to_day_mm for x in d1]
        d3_sf = [x*to_day_mm for x in d2] 
        sf_annual = np.reshape(d1,(-1,365)).mean(axis=1)*to_day_year_mm

        #axs.plot(d3,color=Colors[i],label=Labels[i], linestyle="dashed")
        #axs.legend(fontsize=8)
        print (sf_annual)
    return d3_sf, sf_annual

In [14]:
EvapDaily, EvapChange = Evaporation()
#print (EvapAnnual, EvapDaily)

getSurfaceData:  surface_star-evaporative_flux True
/Users/ajc/FUSE/simulations/tests/ats-intermediate-ngee-up/single_column/test0/ats_vis_surface_star_data.h5
[]


In [15]:
def water_content(model='3D'):
    print (Simulations[0][0])
    infile = os.path.join(Simulations[0][0],'ats_vis_data.h5')
    if (model =='ISM'):
        infile = os.path.join(Simulations[0][0],'ats_vis_column_0_data.h5')
    d = h5py.File(infile,'r')
    wc_key = 'water_content.cell.0'
    if (model =='ISM'):
        wc_key = 'column_0-water_content.cell.0'
    keys = list(d[wc_key].keys())
    keys = sorted (keys, key=int)
    d1 = []

    factor = 1/(mol_density *surface_area)*to_mm
    for key in keys:
        x = np.sum(np.concatenate((d[wc_key][key]))) * factor
        d1.append(x)
    wc_c = [d1[-1] - d1[0]]
    return d1,wc_c


In [16]:
WC,WC_change = water_content(model='ISM')

/Users/ajc/FUSE/simulations/tests/ats-intermediate-ngee-up/single_column/test0


In [17]:
def water_content_sf(model='3D'):
    to_day_mm = 86400 * 1000
    to_year_mm = 86400 * 365 * 1000
    
    to_mm = 1000.
    sf_change = -10101
    d3_sf = []
    sf_change = 0
    for i,sim in enumerate(Simulations):
                
        
        if (model =='3D'):
            var_wc_sf = 'surface-water_content'
            d1_sf = getSurfaceData(i, var_wc_sf,average=False)[var_wc_sf]
        else:
            var_wc_sf = 'surface_star-water_content'
            d1_sf = getSurfaceData(i, var_wc_sf,hdf_file='ats_vis_surface_star_data.h5',average=False)[var_wc_sf]        
        
        d2_sf = np.reshape(d1_sf,(-1,365))
        mol_density = 55000.
        basin_area = 1
        factor = 1/(mol_density *surface_area)*to_mm
        d3_sf = [x*factor for x in d2_sf]
        sf_change = [(d2_sf[0][-1] - d2_sf[0][0])*factor]
        
    return d3_sf[0], sf_change

In [18]:
WC_sf, WC_sf_change = water_content_sf(model ='ISM')


getSurfaceData:  surface_star-water_content False
/Users/ajc/FUSE/simulations/tests/ats-intermediate-ngee-up/single_column/test0/ats_vis_surface_star_data.h5


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def water_content_snow(model='3D'):
    sf_change = -10101
    d3_sf = []
    sf_change = 0
    for i,sim in enumerate(Simulations):
        
        if (model =='3D'):
            var_wc_sf = 'snow-water_equivalent'
            d1_sf = getSurfaceData(i, var_wc_sf,hdf_file='ats_vis_snow_data.h5',average=False)[var_wc_sf]
        else:
            var_wc_sf = 'snow_column_0-snow_water_equivalent'
            d1_sf = getSurfaceData(i, var_wc_sf,hdf_file='ats_vis_snow_column_0_data.h5',average=False)[var_wc_sf]  
            
                
        
        d2_sf = np.reshape(d1_sf,(-1,365))
        mol_density = 55000.
        
        factor = 1/(mol_density *surface_area)*to_mm
        d3_sf = [x*factor for x in d2_sf]
        sf_change = [(d2_sf[0][-1] - d2_sf[0][0])*1000/surface_area]
        
    return d3_sf, sf_change

In [ ]:
WC_snow, WC_snow_change = water_content_snow(model='ISM')

In [ ]:
mass_balance = [x + y + z for x,y,z in zip(WC,WC_sf,WC_snow)]
#plt.plot(WC)
#plt.plot(WC_sf)

In [ ]:
print (WC_change, WC_sf_change, EvapChange, WC_snow_change)
print (precip_annual[0], WC_change[0]+ WC_sf_change[0]+ EvapChange[0] + WC_snow_change[0])